## 尝试使用XGBoost来处理

In [37]:
#导入所需的包
import numpy as np
import pandas as pd  # 数据操作

import xgboost as xgb  #算法库
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import gc #内存管理

In [38]:
# 加载数据集
train = pd.read_csv('../../DataSets/ZillowPrize/train_2016_v2.csv')
prop = pd.read_csv('../../DataSets/ZillowPrize/properties_2016.csv')
sample = pd.read_csv('../../DataSets/ZillowPrize/sample_submission.csv')

In [3]:
print "train:\n"
train.head(5)

train:



,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [4]:
print "properties:\n"
prop.head(2)

properties:



,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN


In [5]:
print "sample:\n"
sample.head(3)

sample:



,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0


In [6]:
prop.columns
prop.dtypes

parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64


In [7]:
# convert to float32
for col,dtype in zip(prop.columns,prop.dtypes):
    if dtype == np.float64:
        prop[col] = prop[col].astype(np.float32)

In [8]:
prop.columns
prop.dtypes

parcelid                          int64
airconditioningtypeid           float32
architecturalstyletypeid        float32
basementsqft                    float32
bathroomcnt                     float32
bedroomcnt                      float32
buildingclasstypeid             float32
buildingqualitytypeid           float32
calculatedbathnbr               float32
decktypeid                      float32
finishedfloor1squarefeet        float32
calculatedfinishedsquarefeet    float32
finishedsquarefeet12            float32
finishedsquarefeet13            float32
finishedsquarefeet15            float32
finishedsquarefeet50            float32
finishedsquarefeet6             float32
fips                            float32
fireplacecnt                    float32
fullbathcnt                     float32
garagecarcnt                    float32
garagetotalsqft                 float32
hashottuborspa                   object
heatingorsystemtypeid           float32
latitude                        float32


In [9]:
# 将prop和train进行左连接合并
# link: http://pandas.pydata.org/pandas-docs/stable/merging.html

data_train = pd.merge(train,prop,how='left',on='parcelid')
data_train.head(5)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13


In [10]:
#训练集
X_train = data_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'],axis=1)
print "X_train shape:",X_train.shape
X_train.head(5)


X_train shape: (90275, 55)


,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,NaN,NaN,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13
1,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN
2,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,NaN,NaN,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13
3,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,NaN,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13
4,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13


In [11]:
#测试集
Y_train = data_train['logerror'].values  #.values 转换成list形式
Y_train

array([ 0.0276, -0.1684, -0.004 , ..., -0.2679,  0.0602,  0.4207])

In [12]:
print "Y_train shape:",Y_train.shape


Y_train shape: (90275,)


In [13]:
# 统计object类型的数据
for i in X_train.dtypes[X_train.dtypes==object].index:
    print X_train[i].head(100)
    print X_train[i].describe(),"\n"


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23    True
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
      ... 
70     NaN
71     NaN
72     NaN
73     NaN
74     NaN
75     NaN
76     NaN
77     NaN
78     NaN
79     NaN
80     NaN
81     NaN
82     NaN
83     NaN
84     NaN
85     NaN
86     NaN
87     NaN
88     NaN
89    True
90     NaN
91     NaN
92     NaN
93     NaN
94     NaN
95     NaN
96     NaN
97     NaN
98     NaN
99     NaN
Name: hashottuborspa, Length: 100, dtype: object
count     2365
unique       1
top       True
freq      2365
Name: hashottuborspa, dtype: object 

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16    

In [14]:
# LabelEncoder:对标签进行编码
# 对NaN值进行-1处理
for i in X_train.columns:
    X_train[i] = X_train[i].fillna(-1)
    if X_train[i].dtype == 'object':
        le = LabelEncoder()
        le.fit(X_train[i].values)
        X_train[i] = le.transform(X_train[i].values)

X_train.head(10)

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,1.0,-1.0,-1.0,2.0,3.0,-1.0,4.0,2.0,-1.0,-1.0,...,-1.0,0,122754.0,360170.0,2015.0,237416.0,6735.879883,0,-1.0,6.037107e+13
1,-1.0,-1.0,-1.0,3.5,4.0,-1.0,-1.0,3.5,-1.0,-1.0,...,-1.0,0,346458.0,585529.0,2015.0,239071.0,10153.019531,0,-1.0,-1.000000e+00
2,1.0,-1.0,-1.0,3.0,2.0,-1.0,4.0,3.0,-1.0,-1.0,...,-1.0,0,61994.0,119906.0,2015.0,57912.0,11484.480469,0,-1.0,6.037464e+13
3,1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,2.0,-1.0,-1.0,...,-1.0,0,171518.0,244880.0,2015.0,73362.0,3048.739990,0,-1.0,6.037296e+13
4,-1.0,-1.0,-1.0,2.5,4.0,-1.0,-1.0,2.5,-1.0,-1.0,...,2.0,0,169574.0,434551.0,2015.0,264977.0,5488.959961,0,-1.0,6.059042e+13
5,1.0,-1.0,-1.0,4.0,4.0,-1.0,1.0,4.0,-1.0,-1.0,...,-1.0,0,880650.0,2447951.0,2015.0,1567301.0,27126.570312,0,-1.0,6.037621e+13
6,-1.0,-1.0,-1.0,1.0,2.0,-1.0,7.0,1.0,-1.0,-1.0,...,-1.0,0,64549.0,111521.0,2015.0,46972.0,2304.969971,0,-1.0,6.037542e+13
7,-1.0,-1.0,-1.0,2.5,3.0,-1.0,-1.0,2.5,-1.0,853.0,...,2.0,0,107000.0,306000.0,2015.0,199000.0,3745.500000,0,-1.0,6.111003e+13
8,-1.0,-1.0,-1.0,1.0,2.0,-1.0,-1.0,1.0,-1.0,-1.0,...,-1.0,0,66834.0,210064.0,2015.0,143230.0,2172.879883,0,-1.0,6.059042e+13
9,-1.0,-1.0,-1.0,2.0,2.0,-1.0,-1.0,2.0,-1.0,-1.0,...,1.0,0,109977.0,190960.0,2015.0,80983.0,1940.260010,0,-1.0,6.059063e+13


In [15]:
# train and test split
x_train,x_test,y_train,y_test = train_test_split(X_train,Y_train,train_size=0.8, random_state=42)

In [16]:
print "x_train.shape",x_train.shape
print "x_test.shape",x_test.shape
print "y_train.shape",y_train.shape
print "y_test.shape",y_test.shape

train_columns = x_train.columns

x_train.shape (72220, 55)
x_test.shape (18055, 55)
y_train.shape (72220,)
y_test.shape (18055,)


In [17]:
# 不用的变量进行处理
del data_train
del train
gc.collect()

778

In [18]:
# Build Dmatrix for xgboost
xgb_train = xgb.DMatrix(x_train,label=y_train)
xgb_train

In [19]:
xgb_valid = xgb.DMatrix(x_test,y_test)
xgb_valid

In [20]:
del x_train
del y_train
gc.collect()

14

In [21]:
# Run XGBoost
# xgboost params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'silent': 1
}

watchlist = [(xgb_train,'train'),(xgb_valid,'valid')]

clf = xgb.train(dict(xgb_params, silent=1),xgb_train,1000,watchlist,early_stopping_rounds=100, verbose_eval=10)

[0]	train-mae:0.478909	valid-mae:0.479976
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:0.334822	valid-mae:0.335925
[20]	train-mae:0.238098	valid-mae:0.239239
[30]	train-mae:0.173954	valid-mae:0.175098
[40]	train-mae:0.132036	valid-mae:0.133115
[50]	train-mae:0.105393	valid-mae:0.106386
[60]	train-mae:0.089004	valid-mae:0.089946
[70]	train-mae:0.079389	valid-mae:0.080401
[80]	train-mae:0.073992	valid-mae:0.075114
[90]	train-mae:0.070978	valid-mae:0.072151
[100]	train-mae:0.069335	valid-mae:0.070555
[110]	train-mae:0.068398	valid-mae:0.069671
[120]	train-mae:0.067849	valid-mae:0.069199
[130]	train-mae:0.067515	valid-mae:0.068933
[140]	train-mae:0.067295	valid-mae:0.068792
[150]	train-mae:0.067149	valid-mae:0.068706
[160]	train-mae:0.06703	valid-mae:0.06866
[170]	train-mae:0.066935	valid-mae:0.068637
[180]	train-mae:0.066856	valid-mae:0.068623
[190]	train-mae:0.066798	valid-ma

In [22]:
del x_test
del y_test
gc.collect()

21

In [23]:
# 建立测试集

sample['parcelid'] = sample['ParcelId']
sample.head(4)

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid
0,10754147,0,0,0,0,0,0,10754147
1,10759547,0,0,0,0,0,0,10759547
2,10843547,0,0,0,0,0,0,10843547
3,10859147,0,0,0,0,0,0,10859147


In [24]:
prop.head(4)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN


In [25]:
# 找到测试集的prop
data_test = pd.merge(sample,prop,how='left',on='parcelid')
data_test.head(5)

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,10759547,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,10843547,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,10859147,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,10879947,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.169922,NaN,NaN,NaN


In [26]:
# 对NaN值进行处理
# 对object 列进行处理.
for i in data_test.columns:
    data_test[i] = data_test[i].fillna(-1)
    if data_test[i].dtype == 'object':
        le = LabelEncoder()
        le.fit(data_test[i].values)
        data_test[i] = le.transform(data_test[i].values)
data_test.head(5)

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,-1.0,-1.0,...,-1.0,0,-1.0,9.0,2015.0,9.0,-1.000000,0,-1.0,-1.0
1,10759547,0,0,0,0,0,0,10759547,-1.0,-1.0,...,-1.0,0,-1.0,27516.0,2015.0,27516.0,-1.000000,0,-1.0,-1.0
2,10843547,0,0,0,0,0,0,10843547,-1.0,-1.0,...,-1.0,0,650756.0,1413387.0,2015.0,762631.0,20800.369141,0,-1.0,-1.0
3,10859147,0,0,0,0,0,0,10859147,-1.0,-1.0,...,1.0,0,571346.0,1156834.0,2015.0,585488.0,14557.570312,0,-1.0,-1.0
4,10879947,0,0,0,0,0,0,10879947,-1.0,-1.0,...,-1.0,0,193796.0,433491.0,2015.0,239695.0,5725.169922,0,-1.0,-1.0


In [27]:
data_test.shape

(2985217, 65)

In [28]:
sample_test = data_test[train_columns]
sample_test.head(5)

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,0,-1.0,9.0,2015.0,9.0,-1.000000,0,-1.0,-1.0
1,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,0,-1.0,27516.0,2015.0,27516.0,-1.000000,0,-1.0,-1.0
2,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,0,650756.0,1413387.0,2015.0,762631.0,20800.369141,0,-1.0,-1.0
3,-1.0,-1.0,-1.0,0.0,0.0,3.0,7.0,-1.0,-1.0,-1.0,...,1.0,0,571346.0,1156834.0,2015.0,585488.0,14557.570312,0,-1.0,-1.0
4,-1.0,-1.0,-1.0,0.0,0.0,4.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,0,193796.0,433491.0,2015.0,239695.0,5725.169922,0,-1.0,-1.0


In [29]:
del data_test
del prop

gc.collect()

642

In [30]:
xgb_sample = xgb.DMatrix(sample_test)

In [31]:
del sample_test
gc.collect()

12

In [33]:
# predict
sample_pred = clf.predict(xgb_sample)
sample_pred

array([ 1.28477788,  0.113399  ,  0.27293712, ...,  1.37913895,
        1.37913895,  1.37913895], dtype=float32)

In [34]:
submit = pd.read_csv('../../DataSets/ZillowPrize/sample_submission.csv')

In [35]:
submit.head(5)

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [36]:
for i in submit.columns[submit.columns!='ParcelId']:
    submit[i] = sample_pred
print "Writing csv..."
submit.to_csv('xgboost_result_benchmark_v02.csv', index=False, float_format='%.4f')

Writing csv...
